# Upcapacited Facility Location Problem

## Background

* You are deciding on five new potiental sites to build an FC.
* You have a set of customer demands (individual customer orders) that you plan to fulfill from these FCs
* Every customer order incurs a shipping cost. 
* Shipping cost for each cutomer order = (distance from FC to customer destination)/$100
* We assume all customer orders are the same (no product variation)
* Each facility that is opened will incur a one time fixed cost for construction
* We assume facilities are uncapacitited

## Problem statement
* Which five sites should you open to minimize overall costs while still being able to statify customer demand?


# Mathematical Formulation

### <font color=green>  Decision Variables

<div class="alert alert-block alert-warning">
<b>Decision Variables:</b> $x_{ij}$ and $y_{c}$

Let $x_{ij}$ = the number of orders to ship from facility ${i}$ to customer destination ${j}$<br>
Let $y_{i}$ = 1 if facility ${i}$ is selected, 0 otherwise<br>
    
<br>
$x_{ij}\in {\rm I\!R}$<br>
$y_{i}\in (0,1)$<br>
</div>

### <font color=green>  Parameters

<div class="alert alert-block alert-warning">
Let $I$ be the set of possible FC locations to choose from <br>
Let $J$ be the set of all customers <br>
Let $N$ = the total number of facilities to open<br><br>

Let $f_i$ be the setup cost for constructing facility $i$<br>
Let $c_{ij}$ be the cost of shipping an order from facility $i$ to customer $j$<br>
Let $d_{j}$ be the demand from customer j<br>

</div>

### <font color=green>  Objective Function

<div class="alert alert-block alert-warning">
<b>Objective Function:</b> Minimize Z = startup_cost + transportation_cost


$$ Z = \sum_{j \: \in \:I} f_iy_i + \sum_{i \: \in \:I}\sum_{j \: \in \:J}c_{ij}x_{ij}$$ 

</div>

### <font color=green>  Constraints

<div class="alert alert-block alert-warning">
<b>Cosntratint 1:</b> all demand must be met

$$ 
\sum_{i \: \in \:I} x_{ij} = d_j 
\qquad \forall  \enspace j \: \in \: J
$$ 

</div>

<div class="alert alert-block alert-warning">
<b>Cosntratint 2:</b> shipments can only be made from sites that have been selected to open

$$ 
\sum_{j \: \in \:J} x_{ij} \leq M_iy_i
\qquad \forall  \enspace i \: \in \: I; 
$$ 

</div>

<div class="alert alert-block alert-warning">
<b>Cosntratint 3:</b> Exactly N new sites must be opened (5)

$$ 
\sum_{i \: \in \:I} y_i = N 
$$ 


</div>

<div class="alert alert-block alert-warning">
<b>Cosntratint 4:</b>

$$ 
x_{ij} \geq 0 
\qquad \forall  \enspace i \: \epsilon \: I; 
\enspace j \: \epsilon \: J
$$ 

$$ 
y_{j} \in {0,1}
$$ 


</div>

# CPlex/Python LP Model

### Import packages

In [1]:
import pandas as pd
import numpy as np
from docplex.mp.model import Model
import math

### Intializing the data

In [2]:
# reading in external data
facility_df = pd.read_csv('facility_data.csv')
customer_df = pd.read_csv('customer_data.csv')

In [3]:
# caluclating distance
facility_df['key'] = 1
customer_df['key'] = 1
result = pd.merge(facility_df, customer_df, on ='key').drop("key", 1) 
result['temp_x'] = (result['facility_x'] - result['customer_x'])**2 
result['temp_y'] = (result['facility_y'] - result['customer_y'])**2 
result['temp_x_y'] = (result['temp_x'] + result['temp_y'])
result['distance'] = round(np.sqrt(result['temp_x_y']),2)
distance_df = result.copy()[['facility','customer','distance']]
distance_df.head()

,facility,customer,distance
0,1,1,97.08
1,1,2,56.29
2,1,3,98.08
3,1,4,137.93
4,1,5,198.88


In [4]:
# for use in model 
distance = dict([((t.facility, t.customer),t.distance ) for t in distance_df.itertuples()])
setup_cost = dict([((t.facility),t.setup_cost ) for t in facility_df.itertuples()])
demand = dict([((t.customer),t.demand ) for t in customer_df.itertuples()])
fc =  set(facility_df['facility'])
customer =  set(customer_df['customer'])
edges = [(i, j) for i in fc for j in customer]
N = 5 # number of sits to open
M = round(distance_df['distance'].max()+100,0)

### Create the model

In [5]:
m=Model('TSP')

#### <font color=green>  Decision Variables

In [6]:
x = m.continuous_var_dict(edges, name ='assignment')  
y = m.binary_var_dict(fc, name = 'fc')

#### <font color=green>  Objective Function

In [7]:
trans_cost = m.sum(distance[e]*x[e] for e in edges)
startup_cost = m.sum(setup_cost[i]*y[i] for i in fc)
m.minimize(startup_cost + trans_cost)

#### <font color=green>  Constraints

In [8]:
# Constraint 1: all demand must be met
for j in customer:
    m.add_constraint(m.sum(x[(i,j)] for i in fc) == demand[j], ctname='demand_%d'%j)

# Constraint 2: shipments can only be made if the site has been opened
for i in fc:
    m.add_constraint(m.sum(x[(i,j)] for j in customer) <= M*y[i], ctname='lane_%d'%i)
    
# Constraint 3: exactly five sites must be opened
m.add_constraint(m.sum(y[(i)] for i in fc) == N, ctname='num_sites')
    
       

docplex.mp.LinearConstraint[num_sites](fc_1+fc_2+fc_3+fc_4+fc_5+fc_6+fc_7+fc_8+fc_9+fc_10,EQ,5)

In [9]:
print(m.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: TSP

Minimize
 obj: 97.080000000000 assignment_1_1 + 56.290000000000 assignment_1_2
      + 98.080000000000 assignment_1_3 + 137.930000000000 assignment_1_4
      + 198.880000000000 assignment_1_5 + 156.420000000000 assignment_1_6
      + 52.630000000000 assignment_1_7 + 75.130000000000 assignment_1_8
      + 181.960000000000 assignment_1_9 + 164.680000000000 assignment_1_10
      + 183.110000000000 assignment_1_11 + 83.380000000000 assignment_1_12
      + 179.250000000000 assignment_1_13 + 56.730000000000 assignment_1_14
      + 70.090000000000 assignment_1_15 + 115.940000000000 assignment_1_16
      + 28.020000000000 assignment_1_17 + 55.030000000000 assignment_1_18
      + 119.280000000000 assignment_1_19 + 190.420000000000 assignment_1_20
      + 99.040000000000 assignment_1_21 + 47.420000000000 assignment_1_22
      + 25.180000000000 assignment_1_23 + 170.070000000000 assignment_1_24
      + 218.2200000

In [10]:
m.parameters.timelimit=120
m.parameters.mip.strategy.branch=1
m.parameters.mip.tolerances.mipgap=0.15

solution = m.solve(log_output=True)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
CPXPARAM_MIP_Strategy_Branch                     1
CPXPARAM_TimeLimit                               120
CPXPARAM_MIP_Tolerances_MIPGap                   0.14999999999999999
Tried aggregator 1 time.
Reduced MIP has 61 rows, 510 columns, and 1020 nonzeros.
Reduced MIP has 10 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.38 ticks)
Found incumbent of value 1338027.410000 after 0.01 sec. (1.19 ticks)
Probing time = 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 61 rows, 510 columns, and 1020 nonzeros.
Reduced MIP has 10 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.44 ticks)
Probing time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, usin

In [11]:
m.get_solve_status()


<JobSolveStatus.OPTIMAL_SOLUTION: 2>

In [12]:
solution.display()

solution for: TSP
objective: 1039437.600
assignment_2_1 = 31.000
assignment_2_2 = 37.000
assignment_2_3 = 10.000
assignment_2_14 = 9.000
assignment_2_16 = 22.000
assignment_2_17 = 23.000
assignment_2_23 = 4.000
assignment_2_26 = 50.000
assignment_2_27 = 1.000
assignment_2_28 = 11.000
assignment_2_37 = 9.000
assignment_2_41 = 27.000
assignment_2_45 = 8.000
assignment_2_46 = 34.000
assignment_2_49 = 2.000
assignment_4_35 = 1.000
assignment_6_4 = 2.000
assignment_6_5 = 35.000
assignment_6_6 = 5.000
assignment_6_9 = 15.000
assignment_6_10 = 11.000
assignment_6_11 = 26.000
assignment_6_13 = 19.000
assignment_6_20 = 40.000
assignment_6_24 = 41.000
assignment_6_25 = 1.000
assignment_6_30 = 35.000
assignment_6_33 = 10.000
assignment_6_34 = 33.000
assignment_6_36 = 27.000
assignment_6_42 = 6.000
assignment_6_47 = 14.000
assignment_6_48 = 16.000
assignment_6_50 = 19.000
assignment_7_3 = 13.000
assignment_7_7 = 17.000
assignment_7_8 = 20.000
assignment_7_12 = 43.000
assignment_7_15 = 41.000
assig

In [13]:
# Export results to csv
import os

base_dir = os.getcwd()

def export_soln_to_csv(df, model_name = 'untitled'):
    """ model refers to model object from docplex.mp.model"""

    try:
        os.mkdir(os.path.join(base_dir, 'output'))
    except:
        pass

    filename = 'output/' + 'soln_' + model_name + '.csv'
    solution_output = os.path.join(os.getcwd(), filename)
    df.to_csv(solution_output, index=False)